In [2]:
# Data link: https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/447dd4/DIC.zip
!mkdir -p ../data/
!curl -L -o ../data/DIC.zip https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/447dd4/DIC.zip
!unzip -d ../data ../data/DIC.zip
!rm ../data/DIC.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6707k  100 6707k    0     0  2439k      0  0:00:02  0:00:02 --:--:-- 2439k
Archive:  ../data/DIC.zip
   creating: ../data/DIC/
  inflating: ../data/DIC/Allianz.pdf  
  inflating: ../data/DIC/Carmigac.pdf  
  inflating: ../data/DIC/credit_mutuel_arkea.pdf  
  inflating: ../data/DIC/fidelity.pdf  
  inflating: ../data/DIC/fidelity2.pdf  
  inflating: ../data/DIC/FR0007040373_DIC_FR_20230630.pdf  
  inflating: ../data/DIC/FR0007050570_DIC_FR_20230630.pdf  
  inflating: ../data/DIC/FR0010902726_DIC_FR_20230831.pdf  
  inflating: ../data/DIC/FR0011443225_DIC_FR_20230630.pdf  
  inflating: ../data/DIC/FR0011443233_DIC_FR_20230630.pdf  
  inflating: ../data/DIC/FR001400BQ78_DIC_FR_20230630.pdf  
  inflating: ../data/DIC/Independance et expension.pdf  
  inflating: ../data/DIC/JP Morgan.pdf  
  inflating: ../data/DIC/KIDPRIIPs_260056_

# Loading

In [ ]:
!pip install -q \
    langchain-chroma==0.1.4 \
    langchain_huggingface \
    langchain-community pypdf \
    lark==1.2.2

In [ ]:

import glob
from langchain_community.document_loaders import PyPDFLoader
import os

documents = []
DIC_path = "/Users/kraligan/RAG_agent_financial_document/data/DIC/*.pdf"

for file in glob.glob(DIC_path):
    try:
        loader = PyPDFLoader(file)  # Retourne une liste de document (un pour chaque page)
        documents += loader.load()
    except Exception:
        print(f"Erreur survenue pour le fichier '{file}'.")


Erreur survenue pour le fichier '/Users/kraligan/RAG_agent_financial_document/data/DIC/Swisslife.pdf'.


In [ ]:
print(len(documents))
print(documents[0])


Document(metadata={'producer': 'Actuate PDF Writer (Low Resolution) 2.1', 'creator': 'Actuate e.Reports', 'creationdate': '2022-07-29T08:11:45+01:00', 'title': '', 'subject': '', 'author': 'IDS GmbH - Analysis and Reporting Services', 'keywords': 'FR0010032326 (22.08.2022)', 'source': '/Users/kraligan/RAG_agent_financial_document/data/DIC/Allianz.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='\x00I\x00n\x00f\x00o\x00r\x00m\x00a\x00t\x00i\x00o\x00n\x00s\x00 \x00c\x00l\x00Ø\x00s\x00 \x00p\x00o\x00u\x00r\x00 \x00l \x19\x00i\x00n\x00v\x00e\x00s\x00t\x00i\x00s\x00s\x00e\x00u\x00r\n\x00C\x00e\x00 \x00d\x00o\x00c\x00u\x00m\x00e\x00n\x00t\x00 \x00f\x00o\x00u\x00r\x00n\x00i\x00t\x00 \x00d\x00e\x00s\x00 \x00i\x00n\x00f\x00o\x00r\x00m\x00a\x00t\x00i\x00o\x00n\x00s\x00 \x00e\x00s\x00s\x00e\x00n\x00t\x00i\x00e\x00l\x00l\x00e\x00s\x00 \x00a\x00u\x00x\x00 \x00i\x00n\x00v\x00e\x00s\x00t\x00i\x00s\x00s\x00e\x00u\x00r\x00s\x00 \x00d\x00e\x00 \x00c\x00e\x00t\x00 \x00O\x00P\x00C\x00V

# Chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n", "\n\n"],
    chunk_size = 600,
    chunk_overlap = 60,
    length_function = len
)

chunks = text_splitter.split_documents(documents=documents)
print(f"{len(chunks)} chunks ont été créés par le splitter à partir du document PDF.")

# Embedding

In [ ]:
from langchain_huggingface.embedding import HuggingFaceEmbeddings
from langchain_chroma import Chroma

def get_vectorstore(docs, model_name="intfloat/multilingual-e5-large", normalize_embeddings=True):
    encore_kwargs = {"normalize_embeddings": normalize_embeddings}
    embedding = HuggingFaceEmbeddings(model_name=model_name, encore_kwargs=encore_kwargs)
    vectore_store = Chroma.from_documents(documents=docs, embedding=embedding)
    return vectore_store

vector_store = get_vectorstore(chunks)

retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs={'k':3})



In [ ]:
results = retriever.invoke("Question")

for i, result in enumerate(results):
    print("--- ")
    print(f"== Contenu du chunk {i} ==\n{result.page_content}")

# LLM

In [ ]:
!huggingface-cli login --token=...

In [ ]:
import transformers
import torch

from transformers import BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

model_config = transformers.AutoConfig.from_pretrained(model_id)
tokenizer = transformers.AutoConfig.from_pretrained(model_id)
model = transformers.AutoConfig.from_pretrained(
    model_id,
    trust_remote_code = True,
    config = model_config,
    device_map = 'auto'
)

In [ ]:
from langchain_huggingface_llms import HuggingFacePipeline

from transformers import pipeline

llm = HuggingFacePipeline(
    pipeline = pipeline(
        'text-generation',
        model=model, 
        tokenizer=tokenizer,
        max_new_sample = 4096,
        do_sample = False,
        return_full_text = False # Très important ! On ne veut pas le prompt initial
    )
)

# RAG Pipeline

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("You are an assistant for question-answer tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise." \
"Question: {question}" \
"Context: {context}" \
"Answer: ")

def rag_pipeline(query):
    retrieved_docs = retriever.invoke(query)
    prompt = prompt_template.invoke({
        "question": query,
        "context": retrieved_docs
    })
    return llm.invoke(prompt).strip()

In [ ]:
query = """

"""

# Effectuer une requête
response = rag_pipeline(query)
print(response)